In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

class MaintenancePredictiveAI:
    def __init__(self, composant="amortisseur"):
        self.composant = composant
        self.modeles = {}
        self.scalers = {}
        self.seuils_alerte = {
            'critique': 0.9,
            'warning': 0.7, 
            'normal': 0.5
        }
    
    def generer_donnees_synthetiques(self, n_echantillons=10000):
        """Genere des donnees synthetiques pour l'entrainement"""
        data = []
        
        for i in range(n_echantillons):
            # Parametres d'usage
            kilometrage = np.random.uniform(0, 300000)
            age_vehicule = np.random.uniform(0, 20)
            vitesse_moyenne = np.random.uniform(20, 120)
            
            # Facteur d'usure combine
            facteur_usure = (kilometrage / 100000) * \
                          np.exp(age_vehicule / 20) * \
                          np.random.lognormal(0, 0.3)
            
            # Duree de vie restante
            duree_vie_nominale = 150000
            duree_vie_restante = max(0, duree_vie_nominale *(1 - facteur_usure))
            
            # Classification de l'etat
            if duree_vie_restante < 10000:
                etat = 'critique'
            elif duree_vie_restante < 30000:
                etat = 'warning'
            else:
                etat = 'normal'
            
            echantillon = {
                'kilometrage': kilometrage,
                'age_vehicule': age_vehicule,
                'vitesse_moyenne': vitesse_moyenne,
                'facteur_usure': facteur_usure,
                'duree_vie_restante': duree_vie_restante,
                'etat': etat
            }
            data.append(echantillon)
        
        return pd.DataFrame(data)
    
    def entrainer_modeles(self, df):
        """Entraîne les modeles de prediction"""
        # Préparation des données
        features = ['kilometrage', 'age_vehicule', 'vitesse_moyenne']
        X = df[features].values
        y = df['duree_vie_restante'].values
        
        # Normalisation
        self.scalers['features'] = StandardScaler()
        X_scaled = self.scalers['features'].fit_transform(X)
        
        # Entrainement modele de regression
        self.modeles['regression'] = RandomForestRegressor(
            n_estimators=100, random_state=42
        )
        self.modeles['regression'].fit(X_scaled, y)
        
        print("Entraînement terminé!")
    
    def predire(self, donnees_vehicule):
        """Effectue une prédiction pour un véhicule"""
        features = ['kilometrage', 'age_vehicule', 'vitesse_moyenne']
        X = np.array([[donnees_vehicule[f] for f in features]])
        X_scaled = self.scalers['features'].transform(X)
        
        duree_vie_predite = self.modeles['regression'].predict(X_scaled)[0]
        
        # Classification de l'urgence
        if duree_vie_predite < 10000:
            urgence = "CRITIQUE"
        elif duree_vie_predite < 30000:
            urgence = "ATTENTION"  
        else:
            urgence = "NORMAL"
        
        return {
            'duree_vie_restante_km': max(0, duree_vie_predite),
            'urgence': urgence
        }







In [13]:
donnees.to_csv("donnees_synthetiques.csv", index=False)
print("✅ Données exportées sous 'donnees_synthetiques.csv'")

✅ Données exportées sous 'donnees_synthetiques.csv'


In [ ]:
# Exemple d'utilisation
ai_system = MaintenancePredictiveAI()
donnees = ai_system.generer_donnees_synthetiques(5000)
ai_system.entrainer_modeles(donnees)

Entraînement terminé!


In [ ]:
# Test de prédiction
vehicule_test = {
    'kilometrage': 85000,
    'age_vehicule': 5.5,
    'vitesse_moyenne': 65
}
prediction = ai_system.predire(vehicule_test)
print(f"Durée de vie restante: {prediction['duree_vie_restante_km']:.0f} km")
print(f"Urgence: {prediction['urgence']}")

Durée de vie restante: 4504 km
Urgence: CRITIQUE
